# Document Clustering and Topic Modeling

*In* this project, we use unsupervised learning models to cluster unlabeled documents into different groups, visualize the results and identify their latent topics/structures.

## Contents

* [Part 1: Load Data](#Part-1:-Load-Data)
* [Part 2: Tokenizing and Stemming](#Part-2:-Tokenizing-and-Stemming)
* [Part 3: TF-IDF](#Part-3:-TF-IDF)
* [Part 4: K-means clustering](#Part-4:-K-means-clustering)
* [Part 5: Topic Modeling - Latent Dirichlet Allocation](#Part-5:-Topic-Modeling---Latent-Dirichlet-Allocation)


# Part 0: Setup Google Drive Environment

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# https://drive.google.com/open?id=192JMR7SIqoa14vrs7Z9BXO3iK89pimJL
file = drive.CreateFile({'id':'192JMR7SIqoa14vrs7Z9BXO3iK89pimJL'}) # replace the id with id of file you want to access
file.GetContentFile('data.tsv')  

# Part 1: Load Data

In [ ]:
import numpy as np
import pandas as pd
import nltk
import gensim #topic modeling
# REGULAR EXPRESSION
import re


from sklearn import decomposition
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

nltk.download('punkt') #下载stopwords
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Load data into dataframe
df = pd.read_csv('data.tsv', sep='\t', header=0, error_bad_lines=False)

b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: expected 15 fields, saw 22\nSkipping line 197432: expected 15 fields, saw 22\nSkipping line 208016: expected 15 fields, saw 22\nSkipping line 214110: expected 15 fields, saw 22\nSkipping line 244328: expected 15 fields, saw 22\nSkipping line 248519: expected 15 fields,

In [ ]:
df.head() # sentiment analysis -> rating

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31


In [ ]:
# Remove missing value
df.review_body.dropna(inplace=True)

In [ ]:
# use the first 1000 data as our  data , as demo
data = df.loc[:1000, 'review_body'].tolist()

# Part 2: Tokenizing and Stemming

Load stopwords and stemmer function from NLTK library.
Stop words are words like "a", "the", or "in" which don't convey significant meaning.
Stemming is the process of breaking a word down into its root.

In [ ]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english')

print ("We use " + str(len(stopwords)) + " stop-words from nltk library.")
print (stopwords[:10])

We use 179 stop-words from nltk library.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

# tokenization and stemming
def tokenization_and_stemming(text):
    # exclude stop words and tokenize the document, generate a list of string 
    tokens = [word.lower() for word in nltk.word_tokenize(text) if word not in stopwords]

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
            
    # stemming
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

# tokenization without stemming
def tokenization(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word not in stopwords]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [ ]:
# tokenization and stemming
tokenization_and_stemming(data[0])

['absolut',
 'love',
 'watch',
 'get',
 'compliment',
 'almost',
 'everi',
 'time',
 'i',
 'wear',
 'dainti']

Use our defined functions to analyze (i.e. tokenize, stem) our reviews.

In [ ]:
# 1. do tokenization and stemming for all the documents
# 2. also just do tokenization for all the documents
# the goal is to create a mapping from stemmed words to original tokenized words for result interpretation.
docs_stemmed = []
docs_tokenized = []
for i in data:
    tokenized_and_stemmed_results = tokenization_and_stemming(i)
    docs_stemmed.extend(tokenized_and_stemmed_results)
    
    tokenized_results = tokenization(i)
    docs_tokenized.extend(tokenized_results)

In [ ]:
# create a mapping from stemmed words to original words
vocab_frame_dict = {docs_stemmed[x]:docs_tokenized[x] for x in range(len(docs_stemmed))}
print(vocab_frame_dict)

{'absolut': 'absolutely', 'love': 'loved', 'watch': 'watch', 'get': 'get', 'compliment': 'compliments', 'almost': 'almost', 'everi': 'every', 'time': 'time', 'i': 'i', 'wear': 'wear', 'dainti': 'dainty', 'keep': 'keep', 'wonder': 'wonderful', 'scratch': 'scratches', 'it': 'it', 'work': 'work', 'well': 'well', 'howev': 'however', 'found': 'found', 'cheaper': 'cheaper', 'price': 'price', 'place': 'place', 'make': 'make', 'purchas': 'purchased', 'beauti': 'beautiful', 'face': 'face', 'the': 'the', 'band': 'band', 'look': 'looks', 'nice': 'nice', 'around': 'around', 'link': 'links', 'squeaki': 'squeaky', 'cheapo': 'cheapo', 'nois': 'noise', 'swing': 'swing', 'back': 'back', 'forth': 'forth', 'wrist': 'wrist', 'embarrass': 'embarrassing', 'front': 'front', 'enthusiast': 'enthusiasts', 'nake': 'naked', 'eye': 'eyes', 'afar': 'afar', 'ca': 'ca', "n't": "n't", 'tell': 'tell', 'cheap': 'cheap', 'fold': 'folds', 'polish': 'polishing', 'brush': 'brushed', 'pretti': 'pretty', 'tight': 'tight', 'pa

# Part 3: TF-IDF

TF: Term Frequency

IDF: Inverse Document Frequency

***example:***

document1: "Arthur da Jason"

document 2: "Jason da da huang"

document1: tf-idf [1, 0.5, 0.5, 0];  document2: tf-idf [0, 1, 0.5, 1]  

2-gram: 

document 1: Arthur da, da Jason; document 2: Jason da, da da, da huang bigram

3-gram:

document 1: Athur da Jason;  document 2: Jason da da, da da huang

[Arhur, da, Jason...]

In [ ]:
# define vectorizer parameters
# TfidfVectorizer will help us to create tf-idf matrix
# max_df : maximum document frequency for the given word
# min_df : minimum document frequency for the given word
# max_features: maximum number of words
# use_idf: if not true, we only calculate tf
# stop_words : built-in stop words
# tokenizer: how to tokenize the document
# ngram_range: (min_value, max_value), eg. (1, 3) means the result will include 1-gram, 2-gram, 3-gram
tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(1,1))

tfidf_matrix = tfidf_model.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      " reviews and " + str(tfidf_matrix.shape[1]) + " terms.")

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In total, there are 1000 reviews and 245 terms.


In [ ]:
print(tfidf_matrix)

  (0, 233)	0.3384314773121352
  (0, 221)	0.2913457104640176
  (0, 72)	0.48447912757435474
  (0, 49)	0.46320707753263723
  (0, 230)	0.16540744731980545
  (0, 124)	0.26150574089750894
  (0, 3)	0.5055255822169622
  (1, 221)	0.6855151455877585
  (1, 230)	0.389191624445557
  (1, 124)	0.6153038799091247
  (3, 165)	0.382128480198896
  (3, 126)	0.4094324612441284
  (3, 156)	0.5262476537694314
  (3, 162)	0.3362979204153272
  (3, 105)	0.45122907749587593
  (3, 240)	0.30447055612845897
  (4, 57)	0.15318217348217658
  (4, 93)	0.11065612128160672
  (4, 112)	0.22130955741998992
  (4, 81)	0.1748276313072244
  (4, 234)	0.16473514304285583
  (4, 228)	0.17842634892207415
  (4, 95)	0.10774641155392574
  (4, 136)	0.4030645760868654
  (4, 29)	0.29513900050991076
  :	:
  (995, 151)	0.23535263450269214
  (995, 195)	0.22821824487010978
  (995, 210)	0.19250454624278207
  (995, 109)	0.450106621790073
  (995, 193)	0.20203435236685066
  (995, 185)	0.19545102928179944
  (995, 0)	0.17333120006622146
  (995, 219)	0.

In [ ]:
# check the parameters
tfidf_model.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 0.99,
 'max_features': 1000,
 'min_df': 0.01,
 'ngram_range': (1, 1),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': 'english',
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': <function __main__.tokenization_and_stemming>,
 'use_idf': True,
 'vocabulary': None}

Save the terms identified by TF-IDF.

In [ ]:
# words
tf_selected_words = tfidf_model.get_feature_names()

In [ ]:
# print out words
tf_selected_words

["'m",
 "'s",
 'abl',
 'absolut',
 'accur',
 'actual',
 'adjust',
 'alarm',
 'alreadi',
 'alway',
 'amaz',
 'amazon',
 'anoth',
 'arm',
 'arriv',
 'automat',
 'awesom',
 'bad',
 'band',
 'batteri',
 'beauti',
 'best',
 'better',
 'big',
 'bit',
 'black',
 'blue',
 'bought',
 'box',
 'br',
 'bracelet',
 'brand',
 'break',
 'bright',
 'broke',
 'button',
 'buy',
 'ca',
 'came',
 'case',
 'casio',
 'chang',
 'cheap',
 'clasp',
 'classi',
 'clock',
 'color',
 'come',
 'comfort',
 'compliment',
 'cool',
 'cost',
 'crown',
 'crystal',
 'dark',
 'date',
 'daughter',
 'day',
 'deal',
 'definit',
 'deliveri',
 'design',
 'dial',
 'differ',
 'difficult',
 'disappoint',
 'display',
 'dress',
 'durabl',
 'easi',
 'easili',
 'end',
 'everi',
 'everyday',
 'everyth',
 'exact',
 'excel',
 'expect',
 'expens',
 'face',
 'fair',
 'far',
 'fast',
 'featur',
 'feel',
 'fell',
 'fine',
 'finish',
 'fit',
 'function',
 'gave',
 'gift',
 'gold',
 'good',
 'got',
 'great',
 'hand',
 'happi',
 'hard',
 'heavi

# (Optional) Calculate Document Similarity

In [ ]:
# use cosine similarity to check the similarity for two documents
from sklearn.metrics.pairwise import cosine_similarity
cos_matrix = cosine_similarity(tfidf_matrix)
print (cos_matrix)

[[1.         0.42500259 0.         ... 0.         0.0400497  0.        ]
 [0.42500259 1.         0.         ... 0.         0.09423401 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.0400497  0.09423401 0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


# Part 4: K-means clustering

In [ ]:
# k-means clustering
from sklearn.cluster import KMeans

num_clusters = 5

# number of clusters
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

## 4.1. Analyze K-means Result

In [ ]:
# create DataFrame films from all of the input files.
product = { 'review': df[:1000].product_title, 'cluster': clusters}
frame = pd.DataFrame(product, columns = ['review', 'cluster'])

In [ ]:
frame.head(10)

,review,cluster
0,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",3
1,Kenneth Cole New York Women's KC4944 Automatic...,3
2,Ritche 22mm Black Stainless Steel Bracelet Wat...,2
3,Citizen Men's BM8180-03E Eco-Drive Stainless S...,2
4,Orient ER27009B Men's Symphony Automatic Stain...,2
5,Casio Men's GW-9400BJ-1JF G-Shock Master of G ...,3
6,Fossil Women's ES3851 Urban Traveler Multifunc...,0
7,INFANTRY Mens Night Vision Analog Quartz Wrist...,2
8,G-Shock Men's Grey Sport Watch,0
9,Heiden Quad Watch Winder in Black Leather,2


In [ ]:
print ("Number of reviews included in each cluster:")
frame['cluster'].value_counts().to_frame()

Number of reviews included in each cluster:


,cluster
2,659
3,108
0,96
1,76
4,61


In [ ]:
print ("<Document clustering result by K-means>")

#km.cluster_centers_ denotes the importances of each items in centroid.
#We need to sort it in decreasing-order and get the top k items.
order_centroids = km.cluster_centers_.argsort()[:, ::-1]  
#argsort： 选出array里最大值
#cluster_centers_ array，看中心点，5个cluster*245feature/words/tf-idf的值

Cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:", end='')
    Cluster_keywords_summary[i] = []
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        Cluster_keywords_summary[i].append(vocab_frame_dict[tf_selected_words[ind]])
        print (vocab_frame_dict[tf_selected_words[ind]] + ",", end='')
    print ()
    
    cluster_reviews = frame[frame.cluster==i].review.tolist()
    print ("Cluster " + str(i) + " reviews (" + str(len(cluster_reviews)) + " reviews): ")
    print (", ".join(cluster_reviews))
    print ()
  
  # 从每个cluster里拿出前几重要的词
  # 强行看

<Document clustering result by K-means>
Cluster 0 words:great,watch,looks,price,work,comfortable,
Cluster 0 reviews (96 reviews): 
Fossil Women's ES3851 Urban Traveler Multifunction Stainless Steel Watch - Rose, G-Shock Men's Grey Sport Watch, HDE Watch Link Pin Remover Band Strap Repair Tool Kit for Watchmakers with Pack of 3 Extra Pins, August Steiner Men's AS8160TTG Silver And Gold Swiss Quartz Watch with Black Dial and Two Tone Bracelet, Lucien Piccard Men's 11577-RG-02S Stockhorn Silver Textured Dial Brown Leather Watch, Timex Ironman Sleek 50-Lap Full Size, Seiko Men's SNKL43 "Seiko 5" Stainless Steel Automatic Watch, Hadley-Roma Men's MSM881RB-170 17-mm Brown Oil-Tan Leather Watch Strap, Michael Kors Petite Silvertone Darci Watch, Ceramic Bezel Insert for Rolex Sea-dweller Deep Sea, Fossil Fossil Ladies Multi-Function White Ceramic Watch, Casio GD110-2CR 55mm Blue Plastic Band & Case Men's Watch, Casio G-Shock Digital Dial Resin Quartz Men's Watch DW6900WW-7, Game Time Men's NFL

# Part 5: Topic Modeling - Latent Dirichlet Allocation

In [ ]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=5, learning_method = 'online')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# LDA requires integer values
tfidf_model_lda = CountVectorizer(max_df=0.99, max_features=500,
                                 min_df=0.01, stop_words='english',
                                 tokenizer=tokenization_and_stemming, ngram_range=(1,1))

tfidf_matrix_lda = tfidf_model_lda.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix_lda.shape[0]) + \
      " reviews and " + str(tfidf_matrix_lda.shape[1]) + " terms.")

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In total, there are 1000 reviews and 245 terms.


In [ ]:
# document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix_lda)
print(lda_output.shape)
print(lda_output)

(1000, 5)
[[0.48740415 0.02526923 0.43669621 0.02511219 0.02551822]
 [0.0528759  0.05028864 0.79244491 0.05073923 0.05365133]
 [0.2        0.2        0.2        0.2        0.2       ]
 ...
 [0.10022029 0.10000334 0.10000968 0.59952223 0.10024446]
 [0.79756622 0.05019916 0.05101547 0.05043761 0.05078155]
 [0.03371813 0.86527468 0.03387609 0.03333822 0.03379288]]


In [ ]:
# topics and words matrix
topic_word = lda.components_
print(topic_word.shape)
print(topic_word)

(5, 245)
[[32.32432625 97.75142467  0.2245553  ...  9.62927722 28.69973525
   7.7793401 ]
 [ 3.99248665 36.48319404  0.21246842 ...  0.20789636  9.28294405
   0.5146028 ]
 [ 2.74091365 15.95737619  2.18968878 ...  7.59944756  9.20684266
   7.27012677]
 [ 0.20427835  0.75010728  0.20529895 ...  0.21760173  0.59696094
   0.20398933]
 [ 8.89547473 81.56675156 13.54719106 ...  0.24947214 23.31670178
  39.76098737]]


In [ ]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]

# index names
doc_names = ["Doc" + str(i) for i in range(len(data))]

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['topic'] = topic

df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,topic
Doc0,0.49,0.03,0.44,0.03,0.03,0
Doc1,0.05,0.05,0.79,0.05,0.05,2
Doc2,0.20,0.20,0.20,0.20,0.20,0
Doc3,0.03,0.24,0.03,0.03,0.67,4
Doc4,0.28,0.01,0.17,0.01,0.53,4
Doc5,0.04,0.04,0.84,0.04,0.04,2
Doc6,0.03,0.03,0.49,0.42,0.03,2
Doc7,0.03,0.03,0.03,0.03,0.88,4
Doc8,0.18,0.01,0.27,0.08,0.45,4
Doc9,0.02,0.02,0.02,0.02,0.92,4


In [ ]:
df_document_topic['topic'].value_counts().to_frame()

,topic
0,275
4,252
2,228
3,171
1,74


In [ ]:
# topic word matrix
print(lda.components_)
# topic-word matrix
df_topic_words = pd.DataFrame(lda.components_)

# column and index
df_topic_words.columns = tfidf_model_lda.get_feature_names()
df_topic_words.index = topic_names

df_topic_words.head()

[[32.32432625 97.75142467  0.2245553  ...  9.62927722 28.69973525
   7.7793401 ]
 [ 3.99248665 36.48319404  0.21246842 ...  0.20789636  9.28294405
   0.5146028 ]
 [ 2.74091365 15.95737619  2.18968878 ...  7.59944756  9.20684266
   7.27012677]
 [ 0.20427835  0.75010728  0.20529895 ...  0.21760173  0.59696094
   0.20398933]
 [ 8.89547473 81.56675156 13.54719106 ...  0.24947214 23.31670178
  39.76098737]]


,'m,'s,abl,absolut,accur,actual,adjust,alarm,alreadi,alway,amaz,amazon,anoth,arm,arriv,automat,awesom,bad,band,batteri,beauti,best,better,big,bit,black,blue,bought,box,br,bracelet,brand,break,bright,broke,button,buy,ca,came,case,...,star,start,stop,strap,sturdi,style,stylish,super,sure,surpris,swim,tell,thank,thing,think,thought,time,timex,tini,tri,turn,use,valu,ve,want,watch,water,way,wear,week,weight,went,wife,wind,wish,work,worn,worth,wrist,year
Topic0,32.324326,97.751425,0.224555,0.317211,0.279861,15.292538,6.052004,0.351104,10.680527,9.620568,4.884874,6.942333,27.477915,0.211805,1.305001,16.613406,23.609737,7.708793,56.073726,0.210877,4.504434,0.214033,12.449002,15.584053,0.237906,20.119157,23.878932,0.686393,0.206292,1.233762,7.842242,9.484143,10.192459,2.290973,15.904726,0.207537,32.349249,25.792565,25.639343,11.711179,...,30.194654,14.328362,5.119953,43.118472,5.590132,31.036888,13.521421,0.207139,0.269219,5.831222,8.050928,19.142972,0.208289,14.391455,27.166721,9.582305,74.243737,2.775872,0.211527,0.231224,0.636316,5.827377,0.220000,15.278773,24.418348,275.863243,0.207494,8.014139,44.958781,8.595245,8.364675,4.755957,1.439871,9.282098,1.198340,17.470763,5.032839,9.629277,28.699735,7.779340
Topic1,3.992487,36.483194,0.212468,0.203410,0.232586,0.544351,0.209760,13.853164,0.206166,0.204354,0.206888,0.207300,0.202484,4.315633,0.222201,0.205641,0.210796,1.848225,0.213919,0.227518,2.474201,0.234746,0.205340,1.081751,21.195631,0.209772,0.212232,2.616580,0.204531,0.221250,0.224030,2.926254,0.591614,3.811873,0.202516,5.086924,0.206362,0.205346,5.301271,0.209359,...,0.298064,0.202643,0.204030,0.618506,1.520067,0.204302,7.051602,14.399676,0.211005,0.202865,0.211303,0.237969,0.226726,0.207439,0.206306,0.227733,0.226114,12.363548,5.660978,0.202218,13.274261,0.228570,19.499654,0.203566,18.290121,32.549319,0.206411,0.206972,11.237262,0.204846,0.735768,0.205673,0.206620,0.205267,4.094092,3.706608,1.291870,0.207896,9.282944,0.514603
Topic2,2.740914,15.957376,2.189689,16.475291,0.202956,0.204740,0.219569,0.332902,0.206533,0.203739,0.498946,2.262916,0.206008,1.456515,6.561483,0.206696,0.219889,2.028719,5.973657,9.756063,47.792289,0.254480,11.762857,43.486521,0.207932,2.804402,5.715263,31.181492,12.223820,0.206750,4.908446,0.210683,0.562102,0.211119,0.212456,3.003714,7.242133,0.207894,0.258307,4.822666,...,0.204624,0.284391,0.218447,0.220558,0.540619,0.231267,0.207490,0.203982,20.607587,4.653125,0.206528,3.556922,0.209039,0.214196,0.962451,0.216073,10.689437,0.201918,0.296657,0.237915,0.202396,14.812158,0.205442,0.552479,3.193146,222.869960,0.202896,5.456022,0.581419,0.210307,2.804650,0.450849,18.523348,1.617869,5.340077,2.218744,3.648944,7.599448,9.206843,7.270127
Topic3,0.204278,0.750107,0.205299,0.215775,0.205227,0.206308,0.204713,0.202541,0.204934,0.300667,14.429681,0.210949,0.201831,0.203738,0.215418,0.261036,0.213279,0.204920,0.204320,0.891043,13.534833,16.500380,0.205345,0.214266,0.204501,0.205072,0.201963,0.885432,0.209573,0.203207,0.203174,0.214906,0.213352,0.204006,0.202287,0.203212,0.216884,0.202425,0.300939,0.202910,...,0.220429,0.203977,0.204461,0.203605,3.612459,0.202653,0.221588,0.211554,0.211328,1.153479,2.653922,0.202861,18.017709,0.201834,0.204040,0.201927,4.846579,0.205186,0.201189,0.202122,0.201627,0.208858,0.314083,0.202405,0.223532,66.882497,0.266246,0.205113,0.275634,0.270318,0.206764,0.201759,0.206262,0.202772,0.202849,0.212022,0.214612,0.217602,0.596961,0.203989
Topic4,8.895475,81.566752,13.547191,0.224816,12.127747,0.215366,17.834638,2.813598,2.483716,3.428374,0.488359,17.385528,1.228297,10.120044,13.852916,0.269493,0.203078,13.678226,123.613365,36.437972,11.239350,0.230681,5.575938,1.591917,2.652502,2.962177,0.278755,13.825692,2.824626,246.028661,1.237068,0.465372,3.720505,4.391077,9.539654,8.699370,20.873669,0.205972,1.948605,14.404111,...,0.649549,0.613359,22.655664,4.471706,0.670584,0.205790,0.403344,0.202544,0.227075,1.081354,3.023376,0.451922,4.234025,9.963159,8.928239,6.450533,100.072589,6.337408,7.565858

In [ ]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model_lda, lda_model=lda, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,watch,look,n't,like,'s,time,great,band,wear,strap,price,feel,cheap,buy,'m
Topic 1,easi,'s,read,watch,light,love,comfort,function,bit,happi,reason,valu,littl,want,purchas
Topic 2,watch,love,nice,look,beauti,big,color,face,bought,realli,dial,like,differ,husband,size
Topic 3,good,great,watch,product,excel,qualiti,expect,perfect,price,fast,thank,recommend,best,pleas,ship
Topic 4,watch,br,band,work,time,'s,use,day,wear,hand,n't,great,second,good,week
